# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
cities_data = pd.read_csv("../WeatherPy/Output_Data/cities.csv")
cities_data.head()
cities_full = cities_data.dropna().reset_index(drop=True)
cities_full.dtypes

City           object
Latitude      float64
Longitude     float64
Max Temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Country        object
Date           object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)
locations = cities_full[["Latitude", "Longitude"]]
weights = cities_full["Humidity"]

In [32]:
fig = gmaps.figure(
    center = (0.0, 0.0),
    zoom_level = 1.7)
heat_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
vaca_cond = cities_full.loc[
    (cities_full["Max Temp"] < 80) &
    (cities_full["Humidity"] < 40) &
    (cities_full["Cloudiness"] < 25) &
    (cities_full["Wind Speed"] < 5)
]

vaca_cond.reset_index(drop=True)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint george,37.1041,-113.5841,68.00,22.0,1.0,4.61,US,2021-04-16 19:40:55
1,danville,36.5860,-79.3950,68.00,32.0,1.0,4.61,US,2021-04-16 19:40:58
2,bozoum,6.3193,16.3799,79.09,37.0,12.0,2.13,CF,2021-04-16 19:41:35
3,butte,39.6499,-121.5677,73.00,35.0,1.0,3.44,US,2021-04-16 19:41:56
4,beaverlodge,55.2166,-119.4361,60.80,31.0,20.0,3.44,CA,2021-04-16 19:42:36
5,asosa,10.0667,34.5333,73.92,16.0,6.0,2.93,ET,2021-04-16 19:43:03
6,hami,42.8000,93.4500,56.30,11.0,0.0,4.52,CN,2021-04-16 19:43:37
7,athabasca,54.7169,-113.2854,66.20,24.0,18.0,3.44,CA,2021-04-16 19:45:03
8,jiexiu,37.0244,111.9125,42.28,23.0,0.0,4.25,CN,2021-04-16 19:45:31
9,zhangye,38.9342,100.4517,45.50,23.0,5.0,1.23,CN,2021-04-16 19:45:34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [106]:
#Create empty hotel list
hotels = []
hotel_lats = []
hotel_longs = []

#Set base url to use google place nearby search and output json
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set params
radius = 5000
type = "lodging"

    
params = {
    "radius": radius,
    "type": type,
    "key": g_key,
}

for index, row in vaca_cond.iterrows():
    
    #get lat & long
    lat = str(row["Latitude"])
    long = str(row["Longitude"])
    
    #add location to params dictionary
    params["location"] = f'{lat},{long}'
    
    #Make API request
    hotel_data = requests.get(base_url, params).json()
    
    #Add nearest hotel name to list
    try:
        hotels.append(hotel_data["results"][0]["name"])
        hotel_lats.append(hotel_data["results"][0]["geometry"]["location"]["lat"])
        hotel_longs.append(hotel_data["results"][0]["geometry"]["location"]["lng"])
    
    except IndexError:
        hotels.append(None)
        hotel_lats.append(None)
        hotel_longs.append(None)
        

In [117]:
hotel_df = pd.DataFrame({
    "Hotel Name":hotels,
    "Hotel Latitude":hotel_lats,
    "Hotel Longitude":hotel_longs
}).dropna().reset_index(drop=True)
hotel_df

,Hotel Name,Hotel Latitude,Hotel Longitude
0,Best Western Plus Abbey Inn,37.089025,-113.584267
1,Holiday Inn Express Danville,36.594944,-79.404663
2,Meadowbrook Oak's RV and Mobile Home Park,39.641884,-121.592003
3,Side Trax Inn,55.202764,-119.420035
4,Blendana Hotel Assosa,10.059980,34.545627
5,Laoyutang Resort,42.802482,93.491396
6,Athabasca Lodge Motel,54.708014,-113.288084
7,IU Hotel Jinzhong Yuhua Road,37.026034,111.914420
8,Zhangye Lvzhou Holiday Inn,38.933208,100.428128
9,Railway motel,39.263245,55.489600


In [113]:
params = {
    "location": "37.1041,-113.5841",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

hotel_data = requests.get(base_url, params).json()
print(json.dumps(hotel_data, indent=4, sort_keys=True))
print(hotel_data["results"][0]["geometry"]["location"]["lat"])

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLH6yeS4gAk-2MI9s1sBWratpuw_zvuupR_nwRKDDbcSpSU7-T5TJD1cZVpG612vKzcZuNfeknluY5XZ0Q7-artzecivjOVNLY3potFih9K0FvfCM1umFoTIxJokaTnlUN9znefPY5FoTfySMmUszaZrXMY-xHcHaTNuxUhVN4EN6idrWPU8q9bYtrX6H8BI897bzXVp6eHiZN4njUZJjfD0l_s3ZdqSVlqHoD24MJy00adHQX6JQLIqeYiQuee6_Rwm4BR-9OHPVnS3gLVUawgC-M8QSi0MuhmwnZKfWIAfaFXCRPHL_kab0uX6t66Qf0XCAqfUt2tsrmY_MWu_Fsx6DBMJt5boRjr2xxqeTlzc9xvTn6RVk4s591CKD_d0LpOFAcjv2SLR_edJtOQAQEz9VanTFvxMLy0Uotz9RXOf_p5H22hmtXk",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.0890245,
                    "lng": -113.5842669
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.0901987302915,
                        "lng": -113.5824545
                    },
                    "southwest": {
                        "lat": 37.08750076970851,
     

In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [119]:
# Add marker layer ontop of heat map
ideal_hotel_locations = hotel_df[["Hotel Latitude", "Hotel Longitude"]]
marker_layer = gmaps.marker_layer(ideal_hotel_locations)

# Display figure
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))